# Dimensionality reduction for PSF

In [1]:
from psf_constants import PROCESSED_TRAIN_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          PROCESSED_VALIDATION_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TEST_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TRAIN_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          PROCESSED_VALIDATION_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TEST_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TRAIN_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          PROCESSED_VALIDATION_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TEST_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TRAIN_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          PROCESSED_VALIDATION_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TEST_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TRAIN_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          PROCESSED_VALIDATION_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PROCESSED_TEST_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TRAIN_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          LOW_DIMENSION_VALIDATION_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TEST_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TRAIN_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          LOW_DIMENSION_VALIDATION_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TEST_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TRAIN_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          LOW_DIMENSION_VALIDATION_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TEST_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TRAIN_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          LOW_DIMENSION_VALIDATION_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TEST_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TRAIN_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          LOW_DIMENSION_VALIDATION_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSION_TEST_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,\
                          TRAIN_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, \
                          VALIDATION_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TEST_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          PSF_TRAIN_FILE_SUFFIXES, \
                          NUMPY_SUFFIX

import umap
import numpy as np

2024-05-13 22:00:35.754622: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-13 22:00:36.331259: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-13 22:00:36.331493: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-13 22:00:36.404354: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-13 22:00:36.565382: I tensorflow/core/platform/cpu_feature_guar

In [2]:
PATH_DICTIONARY = {
    "TR2": [PROCESSED_TRAIN_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
            LOW_DIMENSION_TRAIN_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX],
    "V2": [PROCESSED_VALIDATION_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
           LOW_DIMENSION_VALIDATION_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TS2": [PROCESSED_TEST_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
            LOW_DIMENSION_TEST_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TR5": [PROCESSED_TRAIN_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
            LOW_DIMENSION_TRAIN_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX],
    "V5": [PROCESSED_VALIDATION_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
           LOW_DIMENSION_VALIDATION_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TS5": [PROCESSED_TEST_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
            LOW_DIMENSION_TEST_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TR9": [PROCESSED_TRAIN_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
            LOW_DIMENSION_TRAIN_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX],
    "V9": [PROCESSED_VALIDATION_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
           LOW_DIMENSION_VALIDATION_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TS9": [PROCESSED_TEST_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
            LOW_DIMENSION_TEST_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TR14": [PROCESSED_TRAIN_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
             LOW_DIMENSION_TRAIN_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX],
    "V14": [PROCESSED_VALIDATION_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
            LOW_DIMENSION_VALIDATION_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TS14": [PROCESSED_TEST_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
             LOW_DIMENSION_TEST_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TR20": [PROCESSED_TRAIN_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
             LOW_DIMENSION_TRAIN_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX],
    "V20": [PROCESSED_VALIDATION_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
            LOW_DIMENSION_VALIDATION_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TS20": [PROCESSED_TEST_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH,
             LOW_DIMENSION_TEST_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
}

In [ ]:
for dataset_name, ds_info in PATH_DICTIONARY.items():
    
    reducer = umap.UMAP(n_neighbors=500,
                        min_dist=0.3,
                        n_components=3,
                        metric='euclidean')
    psf_path = ds_info[0]
    embedding_path = ds_info[1]
    embedding_path = f"{embedding_path}{NUMPY_SUFFIX}"
    
    if dataset_name.startswith("TR"):
        
        print("    Generating UMAP for", psf_path)
        psf_list = []
        for fnumber in PSF_TRAIN_FILE_SUFFIXES:
            psf_subpath = f"{psf_path}{fnumber}{NUMPY_SUFFIX}"
            psf_sublist = np.load(psf_subpath)
            psf_list.append(psf_sublist)
            
        psf_data = np.concatenate(psf_list)
        psf_list = []
        embedding = reducer.fit_transform(psf_data)
        print(embedding.shape)
        np.save(embedding_path, embedding)
        
    else:
        print("    Generating UMAP for", psf_path)
        psf_data = np.load(psf_path)
        embedding = reducer.fit_transform(psf_data)
        np.save(embedding_path, embedding)

    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/ProcessedZernikeComplexFields/TrainData/fc_2M_zernike_complex_fields
(10000, 3)
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/ProcessedZernikeComplexFields/ValidationData/fc_2M_zernike_complex_fields07.npy


# Dimensionality reduction for predicted PSF

In [ ]:
from psf_constants import FC_PREDICTED_TRAIN_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          FC_PREDICTED_TRAIN_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          FC_PREDICTED_TRAIN_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          FC_PREDICTED_TRAIN_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          FC_PREDICTED_TRAIN_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, \
                          LOW_DIMENSIONALITY_PREDICTED_TRAIN_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSIONALITY_PREDICTED_TRAIN_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSIONALITY_PREDICTED_TRAIN_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSIONALITY_PREDICTED_TRAIN_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          LOW_DIMENSIONALITY_PREDICTED_TRAIN_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH, \
                          PSF_TRAIN_FILE_SUFFIXES, \
                          NUMPY_SUFFIX

import umap
import numpy as np

In [ ]:
PATH_DICTIONARY = {
    "TR2": [FC_PREDICTED_TRAIN_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
            LOW_DIMENSIONALITY_PREDICTED_TRAIN_2M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TR5": [FC_PREDICTED_TRAIN_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
            LOW_DIMENSIONALITY_PREDICTED_TRAIN_5M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TR9": [FC_PREDICTED_TRAIN_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
            LOW_DIMENSIONALITY_PREDICTED_TRAIN_9M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TR14": [FC_PREDICTED_TRAIN_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX, 
             LOW_DIMENSIONALITY_PREDICTED_TRAIN_14M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH],
    "TR20": [FC_PREDICTED_TRAIN_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PREFIX,
             LOW_DIMENSIONALITY_PREDICTED_TRAIN_20M_ZERNIKE_COMPLEX_FIELDS_FILE_PATH]
}

In [ ]:
for dataset_name, ds_info in PATH_DICTIONARY.items():
    
    reducer = umap.UMAP(n_neighbors=500,
                        min_dist=0.3,
                        n_components=3,
                        metric='euclidean')
    psf_path = ds_info[0]
    embedding_path = ds_info[1]
    
    if dataset_name.startswith("TR"):
        
        print("    Generating UMAP for", psf_path)
        psf_list = []
        for fnumber in PSF_TRAIN_FILE_SUFFIXES:
            psf_subpath = f"{psf_path}{fnumber}{NUMPY_SUFFIX}"
            psf_sublist = np.load(psf_subpath)
            psf_list.append(psf_sublist)
            
        psf_data = np.concatenate(psf_list)
        psf_list = []
        embedding = reducer.fit_transform(psf_data)
        print(embedding.shape)
        np.save(embedding_path, embedding)
        
    else:
        print("    Generating UMAP for", psf_path)
        psf_data = np.load(psf_path)
        embedding = reducer.fit_transform(psf_data)
        np.save(embedding_path, embedding)

# Dimensionality reduction for output fluxes

In [1]:
from psf_constants import TRAIN_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, \
                          VALIDATION_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TEST_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TRAIN_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, \
                          VALIDATION_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TEST_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TRAIN_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, \
                          VALIDATION_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TEST_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TRAIN_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, \
                          VALIDATION_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TEST_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TRAIN_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, \
                          VALIDATION_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          TEST_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH, \
                          PSF_TRAIN_FILE_SUFFIXES, \
                          NUMPY_SUFFIX

import umap
import numpy as np

2024-05-14 11:00:50.215161: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-14 11:00:50.313764: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-14 11:00:50.313841: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-14 11:00:50.316527: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-14 11:00:50.333997: I tensorflow/core/platform/cpu_feature_guar

In [2]:
PATH_DICTIONARY = {
    "TR2": [TRAIN_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "V2": [VALIDATION_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
           LOW_DIMENSIONALITY_VALIDATION_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TS2": [TEST_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
            LOW_DIMENSIONALITY_TEST_2M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TR5": [TRAIN_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "V5": [VALIDATION_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
           LOW_DIMENSIONALITY_VALIDATION_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TS5": [TEST_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
            LOW_DIMENSIONALITY_TEST_5M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TR9": [TRAIN_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "V9": [VALIDATION_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
           LOW_DIMENSIONALITY_VALIDATION_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TS9": [TEST_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
            LOW_DIMENSIONALITY_TEST_9M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TR14": [TRAIN_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "V14": [VALIDATION_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
           LOW_DIMENSIONALITY_VALIDATION_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TS14": [TEST_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
            LOW_DIMENSIONALITY_TEST_14M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TR20": [TRAIN_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "V20": [VALIDATION_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
           LOW_DIMENSIONALITY_VALIDATION_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
    "TS20": [TEST_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH,
            LOW_DIMENSIONALITY_TEST_20M_ZERNIKE_OUTPUT_FLUXES_FILE_PATH],
}

In [3]:
for dataset_name, ds_info in PATH_DICTIONARY.items():
    
    reducer = umap.UMAP(n_neighbors=500,
                        min_dist=0.1,
                        n_components=2,
                        metric='euclidean')
    output_flux_path = ds_info[0]
    embedding_path = ds_info[1]
    
    if dataset_name.startswith("TR"):
        
        print("    Generating UMAP for", output_flux_path)
        output_fluxes_list = []
        for fnumber in PSF_TRAIN_FILE_SUFFIXES:
            output_flux_subpath = f"{output_flux_path}{fnumber}{NUMPY_SUFFIX}"
            output_flux_sublist = np.load(output_flux_subpath)
            output_fluxes_list.append(output_flux_sublist)
            
        output_fluxes_data = np.concatenate(output_fluxes_list)
        output_fluxes_list = []
        embedding = reducer.fit_transform(output_fluxes_data)
        print(embedding.shape)
        np.save(embedding_path, embedding)
        
    else:
        print("    Generating UMAP for", output_flux_path)
        output_fluxes_data = np.load(output_flux_path)
        embedding = reducer.fit_transform(output_fluxes_data)
        np.save(embedding_path, embedding)

    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/ZernikeOutputFluxes/TrainData/2M_zernike_output_fluxes
(70000, 2)
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/ZernikeOutputFluxes/ValidationData/2M_zernike_output_fluxes07.npy
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/ZernikeOutputFluxes/TestData/2M_zernike_output_fluxes08.npy
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/ZernikeOutputFluxes/TrainData/5M_zernike_output_fluxes
(70000, 2)
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/ZernikeOutputFluxes/ValidationData/5M_zernike_output_fluxes07.npy
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/ZernikeOutputFluxes/TestData/5M_zernike_output_fluxes08.npy
    Generating U

# Dimensionality reduction for LP mode coefficients

In [1]:
from psf_constants import TRAIN_2M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, \
                          VALIDATION_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TEST_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TRAIN_5M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, \
                          VALIDATION_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TEST_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TRAIN_9M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, \
                          VALIDATION_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TEST_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TRAIN_14M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, \
                          VALIDATION_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TEST_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TRAIN_20M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, \
                          VALIDATION_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          TEST_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TRAIN_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_VALIDATION_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          LOW_DIMENSIONALITY_TEST_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH, \
                          PSF_TRAIN_FILE_SUFFIXES, \
                          NUMPY_SUFFIX

import umap
import numpy as np

2024-05-15 09:08:08.931942: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-15 09:08:08.976397: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-15 09:08:08.976426: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-15 09:08:08.977533: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-15 09:08:08.985072: I tensorflow/core/platform/cpu_feature_guar

In [2]:
PATH_DICTIONARY = {
    "TR2": [TRAIN_2M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "V2": [VALIDATION_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
           LOW_DIMENSIONALITY_VALIDATION_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TS2": [TEST_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
            LOW_DIMENSIONALITY_TEST_2M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TR5": [TRAIN_5M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "V5": [VALIDATION_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
           LOW_DIMENSIONALITY_VALIDATION_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TS5": [TEST_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
            LOW_DIMENSIONALITY_TEST_5M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TR9": [TRAIN_9M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "V9": [VALIDATION_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
           LOW_DIMENSIONALITY_VALIDATION_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TS9": [TEST_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
            LOW_DIMENSIONALITY_TEST_9M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TR14": [TRAIN_14M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, 
            LOW_DIMENSIONALITY_TRAIN_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "V14": [VALIDATION_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
            LOW_DIMENSIONALITY_VALIDATION_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TS14": [TEST_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
             LOW_DIMENSIONALITY_TEST_14M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TR20": [TRAIN_20M_ZERNIKE_PSF_LP_MODES_FILE_PREFIX, 
             LOW_DIMENSIONALITY_TRAIN_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "V20": [VALIDATION_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
            LOW_DIMENSIONALITY_VALIDATION_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
    "TS20": [TEST_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH,
             LOW_DIMENSIONALITY_TEST_20M_ZERNIKE_PSF_LP_MODES_FILE_PATH],
}

In [4]:
for dataset_name, ds_info in PATH_DICTIONARY.items():
    
    reducer = umap.UMAP(n_neighbors=500,
                        min_dist=0.1,
                        n_components=2,
                        metric='euclidean')
    lp_coeffs_path = ds_info[0]
    embedding_path = ds_info[1]
    
    if dataset_name.startswith("TR"):
        
        print("    Generating UMAP for", lp_coeffs_path)
        lp_coeffs_list = []
        for fnumber in PSF_TRAIN_FILE_SUFFIXES:
            lp_coeffs_subpath = f"{lp_coeffs_path}{fnumber}{NUMPY_SUFFIX}"
            lp_coeffs_sublist = np.load(lp_coeffs_subpath)
            lp_coeffs_sublist = lp_coeffs_sublist.reshape(10000, 19*2)
            print(lp_coeffs_sublist.shape)
            lp_coeffs_list.append(lp_coeffs_sublist)
            
        lp_coeffs_data = np.concatenate(lp_coeffs_list)
        print(lp_coeffs_data.shape)
        lp_coeffs_list = []
        embedding = reducer.fit_transform(lp_coeffs_data)
        print(embedding.shape)
        np.save(embedding_path, embedding)
        
    else:
        print("    Generating UMAP for", lp_coeffs_path)
        lp_coeffs_data = np.load(lp_coeffs_path)
        lp_coeffs_data = lp_coeffs_data.reshape(10000, 19*2)
        embedding = reducer.fit_transform(lp_coeffs_data)
        np.save(embedding_path, embedding)

    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/LPModesFromZernikePSF/TrainData/2M_lp_modes_from_zernike_psf
(10000, 38)
(10000, 38)
(10000, 38)
(10000, 38)
(10000, 38)
(10000, 38)
(10000, 38)
(70000, 38)
(70000, 2)
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/LPModesFromZernikePSF/ValidationData/2M_lp_modes_from_zernike_psf07.npy
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/LPModesFromZernikePSF/TestData/2M_lp_modes_from_zernike_psf08.npy
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/LPModesFromZernikePSF/TrainData/5M_lp_modes_from_zernike_psf
(10000, 38)
(10000, 38)
(10000, 38)
(10000, 38)
(10000, 38)
(10000, 38)
(10000, 38)
(70000, 38)
(70000, 2)
    Generating UMAP for /home/dani/DaniProjects/SAIL/PhotonicLanternProjects/Data/PSFReconstruction/LPModesFromZernikePSF/Vali